In [1]:
import numpy as np, pymongo,spacy,string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

/home/maaz-lfd/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-08 00:42:52.983443: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-08 00:44:42.549534: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-08 00:44:42.549949: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plu

Initialize tokenier

In [2]:
nlp = spacy.load("en_core_web_sm")

Mongo Connection for corpus

In [3]:
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")

db = mongo_client["cord_literature"]
col_processed_ct = db['processed_clinical_trials']


In [4]:
corpus_id = []; corpus_text =[]; corpus_abstract=[]
for data in col_processed_ct.find({},{'_id':1, 'all_text':1, 'abstract':1}):
    corpus_id.append(data['_id']); corpus_text.append(data['all_text']); corpus_abstract.append(data['abstract'])

In [5]:
print(corpus_id[0])
print(corpus_abstract[0])

PMC137267
recent evidence suggest critically ill patient able tolerate low level haemoglobin previously believe goal transfusing level g l improve mortality clinically important outcome critical care setting question remain laboratory clinical study include recent randomized control trial rct establish transfusing normal haemoglobin concentration improve organ failure mortality critically ill patient addition restrictive transfusion strategy reduce exposure allogeneic transfusion result efficient use red blood cell rbc save blood overall decrease health care cost


In [6]:
# compute the TF-IDF matrix for the corpus
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(corpus_text)
feature_names = vectorizer.get_feature_names()

/home/maaz-lfd/anaconda3/envs/py38/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [7]:
feature_names

['00',
 '000',
 '0000',
 '000001',
 '000005',
 '000007',
 '000009',
 '00001',
 '00002',
 '00009',
 '0001',
 '0001525',
 '0001568',
 '0001776',
 '0001817',
 '0001944',
 '0002',
 '00020',
 '0002520',
 '0002521',
 '0002684',
 '0002697',
 '00026v1',
 '0003',
 '00038469',
 '0004',
 '0005',
 '00055',
 '00059',
 '0006',
 '0006457',
 '000664',
 '0006916',
 '0006950',
 '0006954',
 '0006955',
 '0007',
 '0008',
 '000893',
 '0009128',
 '000b',
 '000c',
 '000e',
 '000kda',
 '000ng',
 '000r',
 '000rpm',
 '000x',
 '000xg',
 '001',
 '0010',
 '0010033',
 '001037',
 '0010941',
 '00116',
 '0012',
 '001210',
 '001236',
 '0012501',
 '00126',
 '00127',
 '0013',
 '0013x',
 '0014',
 '001467',
 '0015',
 '00154',
 '0016',
 '0016853',
 '0017',
 '00171',
 '0018',
 '00181',
 '001810',
 '001863',
 '0019',
 '00198',
 '001984',
 '001__now',
 '001a',
 '001l_s2k_labordiagnostik_schwangerschaftsrelevanter_virusinfektionen_2014',
 '001mfi',
 '001p',
 '001μg',
 '002',
 '0020',
 '0021',
 '00211728',
 '002145',
 '0022',
 '0

In [6]:
tfidf_matrix.toarray()

KeyboardInterrupt: 

In [8]:
# define a function to perform information retrieval
def information_retrieval(query, k=10):
    # preprocess the query
    tok_query = [tok.lemma_.lower().strip() for tok in nlp(str(query)) if not tok.like_num and not tok.is_punct]
    tok_query = [tok for tok in tok_query if (tok not in nlp.Defaults.stop_words and tok not in string.punctuation and type(tok) != int and type(tok) != float)]
    prc_query = ' '.join(tok_query)
    # compute the TF-IDF vector for the query
    query_vector = vectorizer.transform([prc_query])
    query_tfidf = query_vector.toarray()[0]

    # compute the cosine similarity between the query vector and the document vectors
    similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    print("SIMILARITIES ",similarities)
    # rank the documents based on the similarity scores
    ranking = np.argsort(similarities)[::-1][:k]
    return ranking


In [9]:
query = "debate transfusing normal haemoglobin level improve outcome"
ranking = information_retrieval(query)
print(ranking)
print("Top matching documents for query '{}':".format(query))
for rank in ranking:
    print("\t- {}".format(corpus_id[rank]))

SIMILARITIES  [0.29737216 0.03796007 0.0006797  ... 0.00305198 0.00327437 0.        ]
[ 563    0 2011 4668 8330  826 1510  115 1772 2067]
Top matching documents for query 'debate transfusing normal haemoglobin level improve outcome':
	- PMC6864588
	- PMC137267
	- PMC7167525
	- PMC7169263
	- PMC7241107
	- PMC7101844
	- PMC7121298
	- PMC3698065
	- PMC7123871
	- PMC7178832


In [10]:
ranking

array([ 563,    0, 2011, 4668, 8330,  826, 1510,  115, 1772, 2067])

In [13]:
import requests

# Set the URL of the Carrot2 REST API
url = 'http://localhost:8080/service/cluster'

# Set the input data and clustering parameters
data = {
    "documents": [{'title' : text} for text in corpus_abstract[:1000]],
    "algorithm": 'Lingo',
    "language": 'English',
}

# Send a POST request to the Carrot2 REST API
response = requests.post(url, json=data)
print(response.status_code)
# print(response.text)
# Parse the response JSON and print the resulting clusters
clusters = response.json()['clusters']
for i in clusters:
    print('label : ',i['labels'])
    print('no. of docs : ',len(i['documents']))
    print('Confidence score of cluster : ', i['score'])
    # print('docs ', [data['documents'][k] for k in i['documents']])
    print("==========================================")

200
label :  ['Clinical Development']
no. of docs :  167
Confidence score of cluster :  24.54572656553717
label :  ['Group Group']
no. of docs :  144
Confidence score of cluster :  25.76029125312494
label :  ['Immune Response']
no. of docs :  143
Confidence score of cluster :  15.202304757245905
label :  ['Development Vaccine']
no. of docs :  139
Confidence score of cluster :  16.36486172441366
label :  ['Infection Viral']
no. of docs :  136
Confidence score of cluster :  29.3042300790303
label :  ['Day Results']
no. of docs :  135
Confidence score of cluster :  35.75976208569094
label :  ['Patient Severe']
no. of docs :  132
Confidence score of cluster :  14.997963688519137
label :  ['P Conclusion']
no. of docs :  127
Confidence score of cluster :  44.1905750234598
label :  ['Influenza Influenza']
no. of docs :  124
Confidence score of cluster :  41.24456158553948
label :  ['Response Result']
no. of docs :  120
Confidence score of cluster :  36.62437754645861
label :  ['Treatment Meth